In [3]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter
import kgbench as kg
import fire, sys
import math

from kgbench import load, tic, toc, d


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors


#
from torch_geometric.utils import to_networkx
import networkx as nx

from src.rgcn_explainer_utils import *
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [4]:
name = 'aifb'
if name == 'aifb':
    data = kg.load('aifb', torch=True) 
    node_idx = 5757
else:
    data = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/data/IMDB/finals/{name}.pt')

data = prunee(data, 2)
data.triples = torch.Tensor(data.triples).to(int)#data.triples.clone().detach()
data.withheld = torch.Tensor(data.withheld).to(int)#data.withheld.clone().detach()
data.training = torch.Tensor(data.training).to(int)#data.training.clone().detach()
get_relations(data)
#d_classes(data)
dict_classes = {key.item(): data.withheld[:, 0][data.withheld[:, 1] == key].tolist() for key in torch.unique(data.withheld[:, 1])}
if name != 'aifb':
    node_idx = dict_classes[0][0]
#110749%data.num_entities_new

from src.rgcn_explainer_utils import *
v = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/masked_adj/masked_ver{node_idx}')
h = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/masked_adj/masked_hor{node_idx}')
model = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/model_{name}_prune_True')

out = model.forward2(h,v)
#print(out)
#res = nn.Softmax(dim=0)(model.forward2(h,v)[5757, :])
# da.training = torch.tensor(da.training, dtype=torch.int32)
# da.withheld = torch.tensor(da.withheld, dtype=torch.int32)
# idxw, clsw = da.withheld[:, 0], da.withheld[:, 1]
# out_val = out[idxw,:]
res = nn.Softmax(dim=0)(out[node_idx])
print('ypred explain', res)
print(v)
if node_idx in data.withheld[:,0]:
    print('ypred true', data.withheld[data.withheld[:,0]==node_idx,1])
    
from r_exp import hor_ver_graph
model.eval()
hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
y_full = model.forward2(hor_graph, ver_graph)
node_pred_full = y_full[node_idx, :]
res_full = nn.Softmax(dim=0)(node_pred_full)
print('ypred full', res_full)

loaded data aifb (0.1963s).
['abstract', 'address', 'author', 'booktitle', 'carriedOutBy', 'carriesOut', 'chapter', 'dealtWithIn', 'edition', 'editor', 'fax', 'financedBy', 'finances', 'hasProject', 'head', 'homepage', 'howpublished', 'isAbout', 'isWorkedOnBy', 'isbn', 'journal', 'member', 'month', 'name', 'note', 'number', 'pages', 'phone', 'photo', 'projectInfo', 'publication', 'publishes', 'series', 'title', 'type', 'volume', 'worksAtProject', 'year', 'type', 'type', 'range', 'subClassOf', 'allValuesFrom', 'inverseOf', 'onProperty']


{0: ['abstract', 'http://swrc.ontoware.org/ontology#abstract'],
 1: ['address', 'http://swrc.ontoware.org/ontology#address'],
 2: ['author', 'http://swrc.ontoware.org/ontology#author'],
 3: ['booktitle', 'http://swrc.ontoware.org/ontology#booktitle'],
 4: ['carriedOutBy', 'http://swrc.ontoware.org/ontology#carriedOutBy'],
 5: ['carriesOut', 'http://swrc.ontoware.org/ontology#carriesOut'],
 6: ['chapter', 'http://swrc.ontoware.org/ontology#chapter'],
 7: ['dealtWithIn', 'http://swrc.ontoware.org/ontology#dealtWithIn'],
 8: ['edition', 'http://swrc.ontoware.org/ontology#edition'],
 9: ['editor', 'http://swrc.ontoware.org/ontology#editor'],
 10: ['fax', 'http://swrc.ontoware.org/ontology#fax'],
 11: ['financedBy', 'http://swrc.ontoware.org/ontology#financedBy'],
 12: ['finances', 'http://swrc.ontoware.org/ontology#finances'],
 13: ['hasProject', 'http://swrc.ontoware.org/ontology#hasProject'],
 14: ['head', 'http://swrc.ontoware.org/ontology#head'],
 15: ['homepage', 'http://swrc.ontoware

ypred explain tensor([0.2610, 0.2667, 0.2997, 0.1726], grad_fn=<SoftmaxBackward0>)
tensor(indices=tensor([[ 23430,  23444,  23490,  23546,  24301,  24407,  24427,
                         24475,  24503,  24543,  88607, 196312, 229452, 254307,
                        254307, 254307, 254307, 254307, 254307, 254307, 254307,
                        254307, 254307, 328872],
                       [  5757,   5757,   5757,   5757,   5757,   5757,   5757,
                          5757,   5757,   5757,    908,   2227,   1002,   6860,
                          6874,   6920,   6976,   7731,   7837,   7857,   7905,
                          7933,   7973,   5230]]),
       values=tensor([0.9867, 0.9847, 0.9765, 0.1670, 0.9590, 0.1789, 0.2049,
                      0.1637, 0.4731, 0.2031, 0.2127, 0.9915, 0.1836, 0.0320,
                      0.0159, 0.0189, 0.0182, 0.0175, 0.0972, 0.0206, 0.0208,
                      0.0203, 0.0197, 0.4484]),
       size=(753935, 8285), nnz=24, layout=torch.sparse

RGCN()

ypred full tensor([1.2879e-20, 7.0437e-17, 1.0000e+00, 1.2028e-17],
       grad_fn=<SoftmaxBackward0>)


In [9]:
data.i2r[30]
p = [10, 23, 27, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
        30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
        30, 30, 30, 30, 30, 30, 39,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  9,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2]
a = dict(Counter(p))
for val in a.values():
    print(val/len(p))
    


0.012195121951219513
0.012195121951219513
0.012195121951219513
0.47560975609756095
0.012195121951219513
0.4634146341463415
0.012195121951219513


In [14]:
import torch

# Define the sparse tensor
x_indices = torch.tensor([[0, 0], [1, 1], [2, 2], [3, 3], [4, 4]])
x_values = torch.tensor([0.1, 0.9, 0.6, 0.4, 0.7])
x = torch.sparse_coo_tensor(x_indices.t(), x_values, size=(5, 5))

# Convert the sparse tensor to a dense tensor
x_dense = x.to_dense()
print(x_dense)

# Select the subset of the tensor based on the threshold value
subset_mask = (x_dense > 0.5)
print(subset_mask)
subset_values = torch.masked_select(x_values, subset_mask)

# Get the indices of the subset
subset_indices = torch.nonzero(subset_mask)

# Create a new sparse tensor from the subset indices and values
subset_sparse = torch.sparse_coo_tensor(subset_indices.t(), subset_values, size=(5, 5))

# Print the original and subset sparse tensors
print("Original sparse tensor:")
print(x)
print("Subset sparse tensor:")
print(subset_sparse)


tensor([[0.1000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.9000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.7000]])
tensor([[False, False, False, False, False],
        [False,  True, False, False, False],
        [False, False,  True, False, False],
        [False, False, False, False, False],
        [False, False, False, False,  True]])
Original sparse tensor:
tensor(indices=tensor([[0, 1, 2, 3, 4],
                       [0, 1, 2, 3, 4]]),
       values=tensor([0.1000, 0.9000, 0.6000, 0.4000, 0.7000]),
       size=(5, 5), nnz=5, layout=torch.sparse_coo)
Subset sparse tensor:
tensor(indices=tensor([[1, 2, 4],
                       [1, 2, 4]]),
       values=tensor([0.9000, 0.6000, 0.7000]),
       size=(5, 5), nnz=3, layout=torch.sparse_coo)


In [204]:
import torch

# Define the sparse tensor
x_indices = torch.tensor([[  5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   7146,   7188,   7268,   7271,   7275,   7285,
                          7286,   7328,   7335,   7336,   7350,   7411,   7436,
                          7494,   7520,   7521,   7525,   7526,   7527,   7538,
                          7541,   7561,   7562,   7570,   7577,   7579,   7580,
                          7590,   7597,   7601,   7603,   7604,   7606,   7608,
                          7609,   7625,   7627,   7640,   7641],
                       [ 82850, 194808, 223811, 255696, 255738, 255818, 255821,
                        255825, 255835, 255836, 255878, 255885, 255886, 255900,
                        255961, 255986, 256044, 256070, 256071, 256075, 256076,
                        256077, 256088, 256091, 256111, 256112, 256120, 256127,
                        256129, 256130, 256140, 256147, 256151, 256153, 256154,
                        256156, 256158, 256159, 256175, 256177, 256190, 256191,
                        328345,  22247,  22247,  22247,  22247,  22247,  22247,
                         22247,  22247,  22247,  22247,  80242,  22247,  22247,
                         22247,  22247,  22247,  22247,  22247,  22247,  22247,
                         22247,  22247,  22247,  22247,  22247,  22247,  22247,
                         22247,  22247,  22247,  22247,  22247,  22247,  22247,
                         22247,  22247,  22247,  22247,  22247]])
x_values = torch.tensor([0.7860, 0.7742, 0.7578, 0.6618, 0.7514, 0.6915, 0.7297,
                      0.6790, 0.7073, 0.7786, 0.7188, 0.6859, 0.7081, 0.7135,
                      0.7068, 0.4538, 0.7784, 0.7261, 0.7155, 0.7443, 0.7072,
                      0.7629, 0.4549, 0.7795, 0.7685, 0.7690, 0.7494, 0.7700,
                      0.7239, 0.7323, 0.7233, 0.7566, 0.6865, 0.7035, 0.7241,
                      0.7804, 0.7407, 0.7178, 0.7403, 0.7066, 0.6806, 0.7605,
                      0.7032, 0.7122, 0.6902, 0.7911, 0.6915, 0.7158, 0.7021,
                      0.7104, 0.7334, 0.7469, 0.7158, 0.7660, 0.7053, 0.7079,
                      0.6859, 0.7322, 0.7291, 0.7513, 0.7280, 0.7838, 0.6932,
                      0.7714, 0.7731, 0.7565, 0.6880, 0.7127, 0.7472, 0.7469,
                      0.7646, 0.7326, 0.7528, 0.7087, 0.6976, 0.7492, 0.6750,
                      0.7049, 0.7700, 0.7456, 0.7249, 0.7682])
x = torch.sparse_coo_tensor(indices = x_indices, values = x_values)

# Select the subset of the tensor based on the threshold value
subset_mask = (x._values() > 0.5).to_dense()
subset_indices = x._indices()[:, subset_mask.nonzero().squeeze()]
subset_values = x._values()[subset_mask]

# Create a new sparse tensor from the subset indices and values
subset_sparse = torch.sparse_coo_tensor(subset_indices, subset_values)

# Print the original and subset sparse tensors
#print("Original sparse tensor:")
#print(x)
# print("Subset sparse tensor:")
# print(subset_sparse)

sparse_tensor = subset_sparse

_, p = torch.div(sparse_tensor.coalesce().indices(), sparse_tensor.size()[0], rounding_mode='floor')
print("p:", p)

output_indices = sparse_tensor.coalesce().indices()[:, p==10]
output_values = sparse_tensor.coalesce().values()[p==10]

print(output_values)
print(output_indices)
print(dict(Counter(p.tolist())))




p: tensor([10, 25, 29, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 42,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 10,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2])
tensor([0.7860, 0.7660])
tensor([[ 5677,  7350],
        [82850, 80242]])
{10: 2, 25: 1, 29: 1, 33: 37, 42: 1, 2: 38}


In [7]:
x_indices = torch.tensor([[ 88527, 196232, 229372, 254227, 254227, 254227, 254227,
                        254227, 254227, 254227, 254227, 254227, 254227, 254227,
                        254227, 254227, 254227, 254227, 254227, 254227, 254227,
                        254227, 254227, 254227, 254227, 254227, 254227, 254227,
                        254227, 254227, 254227, 254227, 254227, 254227, 254227,
                        254227, 254227, 254227, 254227, 254227, 254227, 254227,
                        328792,  23716,  23758,  23838,  23841,  23845,  23855,
                         23856,  23898,  23905,  23906,  81915,  23981,  24006,
                         24064,  24090,  24091,  24095,  24096,  24097,  24108,
                         24111,  24131,  24132,  24140,  24147,  24149,  24150,
                         24160,  24167,  24171,  24173,  24174,  24176,  24178,
                         24179,  24195,  24197,  24210,  24211],
                       [     0,   4253,    116,   7146,   7188,   7268,   7271,
                          7275,   7285,   7286,   7328,   7335,   7336,   7350,
                          7411,   7436,   7494,   7520,   7521,   7525,   7526,
                          7527,   7538,   7541,   7561,   7562,   7570,   7577,
                          7579,   7580,   7590,   7597,   7601,   7603,   7604,
                          7606,   7608,   7609,   7625,   7627,   7640,   7641,
                          5230,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677]])
x_values = torch.tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
x_ver = torch.sparse_coo_tensor(indices = x_indices, values = x_values)
torch.div(x_ver.coalesce().indices(),data.num_entities, rounding_mode='floor')

tensor([[ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  9, 10, 23, 27, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
         30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
         30, 30, 30, 30, 30, 30, 30, 30, 30, 39],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [6]:
x_indices=torch.tensor([[  5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   7146,   7188,   7268,   7271,   7275,   7285,
                          7286,   7328,   7335,   7336,   7350,   7411,   7436,
                          7494,   7520,   7521,   7525,   7526,   7527,   7538,
                          7541,   7561,   7562,   7570,   7577,   7579,   7580,
                          7590,   7597,   7601,   7603,   7604,   7606,   7608,
                          7609,   7625,   7627,   7640,   7641],
                       [ 82850, 194808, 223811, 255696, 255738, 255818, 255821,
                        255825, 255835, 255836, 255878, 255885, 255886, 255900,
                        255961, 255986, 256044, 256070, 256071, 256075, 256076,
                        256077, 256088, 256091, 256111, 256112, 256120, 256127,
                        256129, 256130, 256140, 256147, 256151, 256153, 256154,
                        256156, 256158, 256159, 256175, 256177, 256190, 256191,
                        328345,  22247,  22247,  22247,  22247,  22247,  22247,
                         22247,  22247,  22247,  22247,  80242,  22247,  22247,
                         22247,  22247,  22247,  22247,  22247,  22247,  22247,
                         22247,  22247,  22247,  22247,  22247,  22247,  22247,
                         22247,  22247,  22247,  22247,  22247,  22247,  22247,
                         22247,  22247,  22247,  22247,  22247]])
x_values=torch.tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

x_hor = torch.sparse_coo_tensor(indices = x_indices, values = x_values)
torch.div(x_ver.coalesce().indices(),data.num_entities, rounding_mode='floor')

tensor([[ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  9, 10, 23, 27, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
         30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
         30, 30, 30, 30, 30, 30, 30, 30, 30, 39],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

tensor([[5677,   10,    0],
        [5677,   23, 4253],
        [5677,   27,  116],
        [5677,   30, 7146],
        [5677,   30, 7188],
        [5677,   30, 7268],
        [5677,   30, 7271],
        [5677,   30, 7275],
        [5677,   30, 7285],
        [5677,   30, 7286],
        [5677,   30, 7328],
        [5677,   30, 7335],
        [5677,   30, 7336],
        [5677,   30, 7350],
        [5677,   30, 7411],
        [5677,   30, 7436],
        [5677,   30, 7494],
        [5677,   30, 7520],
        [5677,   30, 7521],
        [5677,   30, 7525],
        [5677,   30, 7526],
        [5677,   30, 7527],
        [5677,   30, 7538],
        [5677,   30, 7541],
        [5677,   30, 7561],
        [5677,   30, 7562],
        [5677,   30, 7570],
        [5677,   30, 7577],
        [5677,   30, 7579],
        [5677,   30, 7580],
        [5677,   30, 7590],
        [5677,   30, 7597],
        [5677,   30, 7601],
        [5677,   30, 7603],
        [5677,   30, 7604],
        [5677,   30,

In [36]:
result = []
indexes = x_hor.coalesce().indices()%data.num_entities
for j in range(indexes.size()[1]):
    for triple in data.triples:
        if triple[0] == indexes[0][j] and triple[2] == indexes[1][j]:
            result.append(triple)
print(torch.stack(result))


_,p = torch.div(x_hor.coalesce().indices(), data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
s,o = x_hor.coalesce().indices()%data.num_entities
result = torch.stack([s,p,o], dim=1)
print(result)

tensor([[5677,   10,    0],
        [5677,   23, 4253],
        [5677,   27,  116],
        [5677,   30, 7146],
        [5677,   30, 7188],
        [5677,   30, 7268],
        [5677,   30, 7271],
        [5677,   30, 7275],
        [5677,   30, 7285],
        [5677,   30, 7286],
        [5677,   30, 7328],
        [5677,   30, 7335],
        [5677,   30, 7336],
        [5677,   30, 7350],
        [5677,   30, 7411],
        [5677,   30, 7436],
        [5677,   30, 7494],
        [5677,   30, 7520],
        [5677,   30, 7521],
        [5677,   30, 7525],
        [5677,   30, 7526],
        [5677,   30, 7527],
        [5677,   30, 7538],
        [5677,   30, 7541],
        [5677,   30, 7561],
        [5677,   30, 7562],
        [5677,   30, 7570],
        [5677,   30, 7577],
        [5677,   30, 7579],
        [5677,   30, 7580],
        [5677,   30, 7590],
        [5677,   30, 7597],
        [5677,   30, 7601],
        [5677,   30, 7603],
        [5677,   30, 7604],
        [5677,   30,

In [113]:
output_indices, output_values, value_indices=select_relation(sparse_tensor,10)

In [25]:

overall_rel_frequency = dict(Counter(data.triples[:,1].tolist()))#.most_common()

overall_rel_frequency_  = {key: round(value/len(data.triples[:,1].tolist()),5) for key, value in overall_rel_frequency.items()}
dict(sorted(overall_rel_frequency_.items(), key=lambda x: x[1], reverse=True))

0.15522

In [93]:
data.entities = np.append(data.triples[:,0].detach().numpy(),(data.triples[:,2].detach().numpy()))
len(d_classes(data))

22

In [140]:
def get_class_entity(node,data):
    ''' Get class of a node'''
    url = data.i2e[int(node)][0]
    try:
        if '#' in url:
            url = url.split('#')[1]
        if 'http' in url:
            return str(url).split('/')[3]

    except IndexError:
        return 'blank' #str(url)
    
def get_class_relation(node,data):
    ''' Get class of a relation node'''
    rel =str(data.i2r[node]).split('/')[-1]
    if '#' in rel:
            rel = rel.split('#')[1]
    return str(rel)


In [149]:


rdf = [i for i in range(data.num_relations) if 'rdf' in data.i2r[i]]
owl = [i for i in range(data.num_relations) if 'owl' in data.i2r[i]]


39
40
41


In [144]:
data.i2r[2]

'http://swrc.ontoware.org/ontology#author'

In [125]:
def get_all_classes(data):
    ''' Get all classes in a graph'''
    return set(list(set([get_class_entity(node) for node in data.entities])))
get_all_classes(data)

{'',
 ' ) aims to position the EU IT\nindustry at the core of this, by developing the essential semantic\nknowledge technologies for realizing the European Knowledge\nSociety.',
 '3.html',
 'BIK',
 'CoM',
 'DesktopDefault.aspx?tabID=20&showPage=StaffProfile&editId=313&editModuleID=50&editShowMenu=true&lang=de',
 'Forschungsgebiete',
 'Forschungsgruppen',
 'Kooperationen',
 None,
 'Personen',
 'Projekte',
 'Publications',
 'Publikationen',
 'WBS',
 'about.html',
 'blank',
 'cgi-bin',
 'eOrganisation',
 'fbe',
 'index.html',
 'index.php',
 'kangal',
 'mitarbeiter',
 'ontoware.org',
 'optrek',
 'padlr',
 'projects',
 'prost',
 'schmitz',
 'semiport',
 'spp',
 'uli',
 'viror',
 'wiki',
 'wim',
 'wwr',
 'ze',
 '~bes',
 '~jbr',
 '~ld111',
 '~lh71',
 '~mer',
 '~sst',
 '~uko'}

In [131]:
#match = match_to_triples(x_hor, data)
len(d_classes(data))

22

In [135]:
def domain_range_freq(data,num_classes):
    dict_domain = {}
    dict_range = {}
    for m in data.triples:


        if int(m[1]) in dict_domain:
            dict_domain[int(m[1]) ].append(get_class_entity(m[0],data))
        else:
            dict_domain[int(m[1]) ] = [get_class_entity(m[0],data)]
        if int(m[1])  in dict_range:
            dict_range[int(m[1]) ].append(get_class_entity(m[2],data))
        else:
            dict_range[int(m[1]) ] = [get_class_entity(m[2],data)]
    for k,v in dict_domain.items():
        dict_domain[k] = len(set(v))/num_classes
    for k,v in dict_range.items():
        dict_range[k] = set(v)
    return dict_domain, dict_range

dict_domain, dict_range = domain_range_freq(data, len(d_classes(data)))
dict_domain

{42: 0.045454545454545456,
 39: 0.2727272727272727,
 41: 0.045454545454545456,
 7: 0.045454545454545456,
 18: 0.045454545454545456,
 23: 0.22727272727272727,
 38: 0.045454545454545456,
 5: 0.045454545454545456,
 14: 0.045454545454545456,
 15: 0.13636363636363635,
 21: 0.09090909090909091,
 31: 0.045454545454545456,
 12: 0.045454545454545456,
 10: 0.045454545454545456,
 27: 0.045454545454545456,
 30: 0.09090909090909091,
 28: 0.045454545454545456,
 36: 0.045454545454545456,
 4: 0.045454545454545456,
 11: 0.045454545454545456,
 17: 0.09090909090909091,
 29: 0.045454545454545456,
 0: 0.045454545454545456,
 2: 0.045454545454545456,
 3: 0.045454545454545456,
 13: 0.045454545454545456,
 33: 0.045454545454545456,
 37: 0.045454545454545456,
 20: 0.045454545454545456,
 22: 0.045454545454545456,
 25: 0.045454545454545456,
 26: 0.045454545454545456,
 1: 0.045454545454545456,
 32: 0.045454545454545456,
 35: 0.045454545454545456,
 9: 0.045454545454545456,
 24: 0.045454545454545456,
 16: 0.045454545

In [66]:
my_dict = {'a': [1, 2, 3], 'b': [4, 5, 6], 'c': [7, 8, 9]}

# Append value to the 'a' key
my_dict['a'].append(10)

# Print updated dictionary
print(my_dict)

{'a': [1, 2, 3, 10], 'b': [4, 5, 6], 'c': [7, 8, 9]}


In [136]:

#LETS work on different mask initializations 


def construct_edge_mask( num_nodes,sparse_tensor,num_entities, init_strategy="normal", const_val=1.0, relation_id = 2):
    """
    Construct edge mask
    """
    data = self.data
    num_entities = data.num_entities
    torch.manual_seed(42)
    mask = nn.Parameter(torch.FloatTensor(num_nodes))
    if init_strategy == "normal":
        std = nn.init.calculate_gain("relu") * math.sqrt(
            2.0 / (num_nodes + num_nodes)
        )
        with torch.no_grad():
            mask.normal_(1.0, std)
    elif init_strategy == "const":
        nn.init.constant_(mask, const_val) 
    elif init_strategy == "zero_out":
        '''initialize the mask with the zero out strategy: we zero out edges belonging to specific relations'''
        std = nn.init.calculate_gain("relu") * math.sqrt(
            2.0 / (num_nodes + num_nodes)
        )
        with torch.no_grad():
            mask.normal_(1.0, std)
        output_indices, output_values, value_indices=select_relation(sparse_tensor,relation_id)
        mask.data[[value_indices]] = 0
    

    elif init_strategy == "overall_frequency":
        '''Initialize the mask with the overall frequency of the relations'''
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        overall_rel_frequency = dict(Counter(data.triples[:,1].tolist()))#.most_common()

        overall_rel_frequency_  = {key: round(value/len(data.triples[:,1].tolist()),5) for key, value in overall_rel_frequency.items()}
        for i in p:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = overall_rel_frequency_[i]
    
    elif init_strategy == "relative_frequency":
        ''' Initialize the mask with the relative frequency of the relations-relative for the node to be explained'''
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        rel_frequency = dict(Counter(p))
        rel_frequency_  = {key: round(value/len(p),5) for key, value in rel_frequency.items()}
        for i in p:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = rel_frequency_[i]

    elif init_strategy == "inverse_relative_frequency":
        ''' Initialize the mask with the relative frequency of the relations-relative for the node to be explained'''
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        rel_frequency = dict(Counter(p))
        rel_frequency_  = {key: 1 - round(value/len(p),5) for key, value in rel_frequency.items()}
        for i in p:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = rel_frequency_[i]
    elif init_strategy == "domain_frequency":
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        dict_domain, dict_range = domain_range_freq(data, len(d_classes(data)))
        for i in p:

            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = dict_domain[i]
    elif init_strategy == "range_frequency":
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        dict_domain, dict_range = domain_range_freq(data, len(d_classes(data)))
        for i in p:
                _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
                mask.data[[value_indices]] = dict_range[i]
    elif init_strategy == "rdf":
        rdf = [i for i in range(data.num_relations) if 'rdf' in data.i2r[i]]
        for i in rdf:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = 0
    elif init_strategy == "owl":
        owl = [i for i in range(data.num_relations) if 'owl' in data.i2r[i]]
        for i in owl:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = 0


        
    return mask

mask = construct_edge_mask(len(x_hor.coalesce().values()),x_hor,data.num_entities, init_strategy="domain_frequency", const_val=1.0)
print(mask)


Parameter containing:
tensor([0.0455, 0.2273, 0.0455, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909,
        0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909,
        0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909,
        0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909,
        0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.0909, 0.2727, 0.0455, 0.0455,
        0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455,
        0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455,
        0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455,
        0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455, 0.0455,
        0.0455], requires_grad=True)


In [138]:
def construct_experimental_mask(num_nodes, masked_hor, init_strategy = "zero_out", const_val = 1.0):
    if init_strategy== "normal" or "const":
        mask = construct_edge_mask(num_nodes,init_strategy, const_val)
    elif init_strategy == "zero_out":
        mask = construct_edge_mask(num_nodes, "const", const_val)
        print('mask',mask)
        output_indices, output_values, value_indices=select_relation(masked_hor,25)
        mask.data[[value_indices]] = 0
    return mask
construct_experimental_mask(x.size()[0],x, init_strategy = "zero_out", const_val = 1.0)

Parameter containing:
tensor([0.0000e+00, 0.0000e+00, 1.1358e-41,  ..., 4.5827e-41, 1.5134e-42,
        4.5827e-41], requires_grad=True)

In [139]:
def subset_sparse(sparse_full, threshold=0.5):
    ''' Select the subset of the tensor based on the threshold value'''
    x_values = sparse_full.coalesce().values()
    x_indices = sparse_full.coalesce().indices()
    x = torch.sparse_coo_tensor(indices = x_indices, values = x_values)
    subset_mask = (x._values() > threshold).to_dense()
    subset_indices = x._indices()[:, subset_mask.nonzero().squeeze()]
    subset_values = x._values()[subset_mask]
    subset_sparse = torch.sparse_coo_tensor(subset_indices, subset_values)
    num_high = subset_sparse._nnz()
    _ ,p = torch.div(subset_sparse.coalesce().indices(), subset_sparse.size()[0], rounding_mode='floor')
    relation_counter = dict(Counter(p.tolist()))
    return subset_sparse, num_high,p, relation_counter

selected_mask, num_high, p,relation_counter = subset_sparse(x)
print(selected_mask, num_high, p,relation_counter)


def select_relation(sparse_tensor,relation_id):
    ''' Select the subset of the tensor based on the relation id'''
    _ ,p = torch.div(sparse_tensor.coalesce().indices(), sparse_tensor.size()[0], rounding_mode='floor')
    #print(p)
    output_indices = sparse_tensor.coalesce().indices()[:, p==relation_id]
    output_values = sparse_tensor.coalesce().values()[p==relation_id]
    value_indices = torch.where(p == relation_id)[0]
    #I can then use the value indices to change the values in the mask??
    return output_indices, output_values, value_indices

output_indices, output_values, value_indices=select_relation(selected_mask,10)





    


tensor(indices=tensor([[  5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   5677,
                          5677,   5677,   5677,   5677,   5677,   5677,   7146,
                          7188,   7268,   7271,   7275,   7285,   7286,   7328,
                          7335,   7336,   7350,   7411,   7436,   7494,   7520,
                          7521,   7525,   7526,   7527,   7538,   7541,   7561,
                          7562,   7570,   7577,   7579,   7580,   7590,   7597,
                          7601,   7603,   7604,   7606,   7608,   7609,   7625,
                          7627,   7640,   7641],
                       [ 82850, 194808, 223811, 255696, 255738, 255818,

In [ ]:
#suppose I have a list of all the inverse relations - then I could mask it by the inverse relations
#I could also mask it by the relations that have the same head and tail


In [156]:
for i in range(data.num_relations):
    if 'inverse' in data.i2r[i]:
        print(data.i2r[i])
for triple in data.triples:
    print(triple[1])
    if triple[1] == torch.tensor(43):
        print(triple)

http://www.w3.org/2002/07/owl#inverseOf
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(42)
tensor(39)
tensor(41)
tensor(41)
tensor(39)
tensor(41)
tensor(41)
tensor(39)
tensor(41)
tensor(39)
tensor(41)
tensor(41)
tensor(41)
tensor(39)
tensor(41)
tensor(39)
tensor(41)
tensor(39)
tensor(41)
tensor(39)
tensor(41)
tensor(39)
tensor(41)
tensor(41)
tensor(41)
tensor(39)
tensor(41)
tensor(41)
tensor(41)
tensor(39)
tensor(41)
tensor(39)
tensor(41)
tensor(39)
tensor(41)
tensor(41)
tensor(41)
tensor(41)
tensor(41)
tensor(41)
tensor(41)
tensor(39)
tensor(41)
tensor(41)
tensor(41)
tensor(41)
tensor(39)
tensor(41)
tensor(41)
tensor(39)
tensor(41)
tensor(39)
tensor(39)
tensor(39)
tensor(39)
tensor(41)
tensor(39)
tensor(7)
tensor(7)
tensor(7)
tensor(7)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(

KeyboardInterrupt: 

In [39]:
import torch

# Define a sparse tensor
indices = torch.tensor([[0, 1, 1], [2, 0, 2]])
values = torch.tensor([3, 4, 5])
sparse_tensor = torch.sparse_coo_tensor(indices, values, size=(3, 3))

# Perform the div operation and obtain a list of integers
_, p = torch.div(sparse_tensor.coalesce().indices(), sparse_tensor.size()[0], rounding_mode='floor')
print("p:", p)

# Create a boolean mask that is True for the elements that match the output integers
mask = p == torch.arange(sparse_tensor.size(0))
print("mask:", mask)

# Get the indices of the elements that match the output integers
indices_subset = sparse_tensor.coalesce().indices()[:, mask]
print("indices_subset:", indices_subset)

# Create a sparse tensor that corresponds to the subset
values_subset = sparse_tensor.coalesce().values()[mask]
sparse_tensor_subset = torch.sparse_coo_tensor(indices_subset, values_subset, size=sparse_tensor.size())
print("sparse_tensor_subset:", sparse_tensor_subset)


p: tensor([0, 0, 0])
mask: tensor([ True, False, False])
indices_subset: tensor([[0],
        [2]])
sparse_tensor_subset: tensor(indices=tensor([[0],
                       [2]]),
       values=tensor([3]),
       size=(3, 3), nnz=1, layout=torch.sparse_coo)


In [ ]:
#create sparse tensor from tensor of indices and values
torch.sparse_coo_tensor(indices, values, size=None, dtype=None, device=None, requires_grad=False)

In [44]:
for node_idx in dict_classes[0]:
    if node_idx in data.withheld[:,0]:
        print('ypred true', data.withheld[data.withheld[:,0]==node_idx,1])
        
    from r_exp import hor_ver_graph
    model.eval()
    hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
    print('num relations', len(hor_graph.coalesce().values()))
    y_full = model.forward2(hor_graph, ver_graph)
    node_pred_full = y_full[node_idx, :]
    res_full = nn.Softmax(dim=0)(node_pred_full)
    print('ypred full', res_full)

ypred true tensor([0])
num relations 26666
ypred full tensor([1.0000e+00, 2.2686e-18, 9.4180e-22, 2.0967e-18],
       grad_fn=<SoftmaxBackward0>)
ypred true tensor([0])
num relations 26666
ypred full tensor([0.7394, 0.1740, 0.0648, 0.0218], grad_fn=<SoftmaxBackward0>)
ypred true tensor([0])
num relations 26666
ypred full tensor([0.5305, 0.2657, 0.1411, 0.0626], grad_fn=<SoftmaxBackward0>)
ypred true tensor([0])
num relations 26666
ypred full tensor([0.4347, 0.3736, 0.1179, 0.0738], grad_fn=<SoftmaxBackward0>)
ypred true tensor([0])
num relations 26666
ypred full tensor([0.4713, 0.3009, 0.1732, 0.0547], grad_fn=<SoftmaxBackward0>)
ypred true tensor([0])
num relations 26666
ypred full tensor([0.4347, 0.3736, 0.1179, 0.0738], grad_fn=<SoftmaxBackward0>)
ypred true tensor([0])
num relations 26666
ypred full tensor([0.5305, 0.2657, 0.1411, 0.0626], grad_fn=<SoftmaxBackward0>)
ypred true tensor([0])
num relations 26666
ypred full tensor([9.9999e-01, 1.2197e-05, 1.1974e-07, 1.1998e-06],
     

KeyboardInterrupt: 

In [29]:
len(hor_graph.coalesce().values())

6098

In [ ]:
d.__dict__.keys()

In [12]:
def selected(masked_ver, threshold,data, low_threshold, float=False):
    sel_masked_ver = sub_sparse_tensor(masked_ver, threshold,data, low_threshold)
    indices_nodes = sel_masked_ver.coalesce().indices().detach().numpy()
    new_index = np.transpose(np.stack((indices_nodes[0], indices_nodes[1]))) 
    triples_matched = match_to_triples(np.array(new_index), data.triples)
    #triples_matched = match_to_triples(sel_masked_ver, data)
    #print(triples_matched)
    if float:
            l = {}
            for i,j in zip(triples_matched[:,1],sel_masked_ver.coalesce().values()):

                if data.i2rel[int(i)][0] in l.keys():
                    l[data.i2rel[int(i)][0]] += np.float(j)
                else:
                    l[data.i2rel[int(i)][0]] = np.float(j)
            return l

    else:
        l = []
        for i in triples_matched[:,1]:
            l.append(data.i2rel[int(i)][0])

        return Counter(l)
    
selected(v, threshold=0.5,data=data, low_threshold=False, float=True)

TypeError: 'bool' object is not callable

In [11]:
float(torch.tensor(2))

2.0

In [ ]:
i = [[0, 1, 1],[2,0,2]]
v =  [[3, 4], [5, 6], [7, 8]]
s = torch.sparse_coo_tensor(i, v, (2, 3, 2))
type(s.coalesce())

In [ ]:
da = kg.load('aifb', torch=True) 
da = prunee(da)
print(f'Number of entities: {da.num_entities}') #data.i2e
print(f'Number of classes: {da.num_classes}')
print(f'Types of relations: {da.num_relations}') #data.i2r
name = 'aifb'
# get_relations(data)
# d_classes(data)
d = {key.item(): da.withheld[:, 0][da.withheld[:, 1] == key].tolist() for key in torch.unique(da.withheld[:, 1])}
#model = torch.load(f'{name}_chk/model_{name}_prune_{prune}')
d

In [ ]:
from src.rgcn_explainer_utils import *
v = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/aifb_chk/masked_adj/masked_ver5905')
h = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/aifb_chk/masked_adj/masked_hor5905')
model = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/aifb_chk/model_aifb_prune_True')
out = model.forward2(h,v)
#print(out)
#res = nn.Softmax(dim=0)(model.forward2(h,v)[5757, :])
# da.training = torch.tensor(da.training, dtype=torch.int32)
# da.withheld = torch.tensor(da.withheld, dtype=torch.int32)
# idxw, clsw = da.withheld[:, 0], da.withheld[:, 1]
# out_val = out[idxw,:]
res = nn.Softmax(dim=0)(out[5905])
print('ypred explain', res)
print(v)
# visualize(node_idx=5757, n_hop = 2, data = data,masked_ver = v, threshold=0.5, name='aifb')
# visualize()

In [ ]:
for target_label in range(len(d.keys())):
    for node_idx in d[target_label]:
        if node_idx not in data.withheld[:, 0]:
            print('node_idx', node_idx)



In [ ]:
from src.rgcn_explainer_utils import *
data = kg.load('aifb', torch=True) 
#data = prunee(data)
data.entities = np.append(data.triples[:,0].detach().numpy(),(data.triples[:,2].detach().numpy()))
get_relations(data)
d_classes(data)
#breakpoint()

v = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/aifb_chk/masked_adj/masked_ver5731')
print(v)
visualize_(5731, 2, data, v, 0.5, 'aifb')

In [ ]:
def sub_sparse_tensor_(sparse_tensor, threshold, data, low_threshold=False):
    if low_threshold:
        nonzero_indices = sparse_tensor.coalesce().indices()
        nonzero_indices[0] = nonzero_indices[0]%data.num_entities
        nonzero_values = sparse_tensor.coalesce().values()
        sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
    else:
        nonzero_indices = sparse_tensor.coalesce().indices()
        nonzero_indices[0] = nonzero_indices[0]%data.num_entities
        nonzero_values = sparse_tensor.coalesce().values()
        sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)    
    return sel_masked_ver


def visualize_(node_idx, n_hop, data, masked_ver,threshold,name, result_weights=False, low_threshold=False ):
    """ 
    Visualize important nodes for node idx prediction
    """
    dict_index = dict_index_classes(data,masked_ver)
    
    #select only nodes with a certain threshold
    sel_masked_ver = sub_sparse_tensor_(masked_ver, threshold,data, low_threshold)
    print('sel masked ver',sel_masked_ver)
    indices_nodes = sel_masked_ver.coalesce().indices().detach().numpy()
    new_index = np.transpose(np.stack((indices_nodes[0], indices_nodes[1]))) #original edge indexes

    
    
    G = nx.Graph()
    if result_weights:
        values = sel_masked_ver.coalesce().values().detach().numpy()
        for s,p,o in zip(indices_nodes[0],values , indices_nodes[1]):
            G.add_edge(int(s), int(o), weight=np.round(p, 2))

    else:
        #get triples to get relations 
        triples_matched = match_to_triples(np.array(new_index), data.triples)
        l = []
        for i in triples_matched[:,1]:
            l.append(data.i2rel[int(i)][0])
        print(Counter(l))
        for s,p,o in triples_matched:
            G.add_edge(int(s), int(o), weight=int(p))

    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())


    pos = nx.circular_layout(G)

    ordered_dict = {}
    for item in list(G.nodes):
        if item in ordered_dict:
            ordered_dict[item].append(dict_index[item])
        # else:
        #     ordered_dict[item] =  dict_index[item]

    dict_index = ordered_dict

    labeldict = {}
    for node in G.nodes:
        labeldict[int(node)] = int(node)  

    print('dict index:', dict_index)

    dict = {}
    for k,v in dict_index.items():
        for k1,v1 in data.entities_classes.items():
            if v==k1: 

                dict[k] = v1
            else:
                if k not in dict:
                    dict[k] = 0
                

    color_list = list(dict.values())
    color_list = list(encode_dict(dict_index).values())


    
    if result_weights:
        
        nx.draw(G, pos,labels = labeldict,  edgelist=edges, edge_color=weights, node_color =  color_list, cmap="Set2",edge_cmap=plt.cm.Reds,font_size=8)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=nx.get_edge_attributes(G,'weight'),font_size=8,font_color='red')
        # sm = plt.cm.ScalarMappable(cmap=plt.cm.Reds, norm=plt.Normalize(vmin=0, vmax=1))
        # sm.set_array(weights)
        # cbar = plt.colorbar(sm)
        # cbar.ax.set_title('Weight')
        plt.title("Node {}'s {}-hop neighborhood important nodes".format(node_idx, n_hop))
    else:
        rel = nx.get_edge_attributes(G,'weight')
        for k,v in rel.items():
            rel[k] = data.i2rel[v][0]
        nx.draw(G, pos,labels = labeldict,  edgelist=edges, edge_color=weights,node_color =  color_list, cmap="Set2",font_size=8)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=rel,font_size=8,font_color='red')
        res = Counter(rel.values())
    if result_weights:
        if not os.path.exists(f'chk/{name}_chk/graphs'):
            os.makedirs(f'chk/{name}_chk/graphs')  
        plt.savefig(f'chk/{name}_chk/graphs/Explanation_{node_idx}_{n_hop}_weights.png')
        plt.show()
    else:
        if not os.path.exists(f'chk/{name}_chk/graphs'):
            os.makedirs(f'chk/{name}_chk/graphs')  
        plt.savefig(f'chk/{name}_chk/graphs/Explanation_{node_idx}_{n_hop}_relations.png')    
        plt.show()
        return res

In [ ]:
v = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/aifb_chk/masked_adj/masked_ver5757')
h = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/aifb_chk/masked_adj/masked_hor5757')
model = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/aifb_chk/model_aifb_prune_True')
res = nn.Softmax(dim=0)(model.forward2(h,v)[5757, :])
print('ypred explain', res)
print(v)
# a = nn.Softmax(dim=0)(model.forward2(h,v))[5757,:]
# a

In [ ]:
v = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/aifb_chk/masked_adj/masked_ver5678')
h = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/aifb_chk/masked_adj/masked_hor5678')
model = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/aifb_chk/model_aifb_prune_True')
res = nn.Softmax(dim=0)(model.forward2(h,v)[5678, :])
print('ypred explain', res)
print(v)
a = nn.Softmax(dim=0)(model.forward2(h,v))[5678,:]
v_val = v.coalesce().values().detach().numpy()
v_val[v_val>0.5]

In [ ]:
from r_exp import hor_ver_graph
model.eval()
hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
y_full = model.forward2(hor_graph, ver_graph)
node_pred_full = y_full[5724, :]
res_full = nn.Softmax(dim=0)(node_pred_full)
print('ypred full', res_full)

In [ ]:
from r_exp import hor_ver_graph

hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
y_full = model.forward2(hor_graph, ver_graph)
node_pred_full = y_full[5757, :]
res_full = nn.Softmax(dim=0)(node_pred_full)
print('ypred full', res_full)
print(hor_graph)

In [ ]:

import torch
name = 'IMDb_us'
data = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/data/IMDB/{name}.pt')
data = prunee(data, 2)
model = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/model_{name}_prune_True')

v = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/IMDb_us_chk/masked_adj/masked_ver5889')
h = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/IMDb_us_chk/masked_adj/masked_hor5889')
res = nn.Softmax(dim=0)(model.forward2(h,v)[5889, :])
print('ypred explain', res)
print(v)

In [ ]:
a = 0
for i in da.triples:
    if 5731 in i:
        #print(i)
        a += 1
a   
da.i2e[5731]

In [ ]:
def prunee(data , n=2):
    """
    Prune a given dataset. That is, reduce the number of triples to an n-hop neighborhood around the labeled nodes. This
    can save a lot of memory if the model being used is known to look only to a certain depth in the graph.

    Note that switching between non-final and final mode will result in different pruned graphs.

    :param data:
    :return:
    """

    data_triples = data.triples
    data_training = data.training
    data_withheld = data.withheld

    if data.torch:
        data_triples = data_triples.numpy()
        data_training = data_training.numpy()
        data_withheld = data_withheld.numpy()

    assert n >= 1

    entities = set()

    for e in data_training[:, 0]:
        entities.add(e)
    for e in data_withheld[:, 0]:
        entities.add(e)

    entities_add = set()
    for _ in range(n):
        for s, p, o in data_triples:
            if s in entities:
                entities_add.add(o)
            if o in entities:
                entities_add.add(s)
        entities.update(entities_add)
    print(entities)
    # new index to old index
    n2o = list(entities)
    o2n = {o: n for n, o in enumerate(entities)}

    nw = Data(dir=None)

    nw.num_entities = len(n2o)
    nw.num_relations = data.num_relations
    nw.i2e = data.i2e
    nw.e2i = data.e2i
    #nw.i2e = [data.i2e[i] for i in range(len(n2o))] #[data.i2e[n2o[i]] for i in range(len(n2o))]
    #nw.e2i = {e: i for i, e in enumerate(nw.i2e)}

    # relations are unchanged, but copied for the sake of GC
    nw.i2r = list(data.i2r)
    nw.r2i = dict(data.r2i)

    # count the new number of triples
    num = 0
    for s, p, o in data_triples:
        if s in entities and o in entities:
            num += 1

    nw.triples = np.zeros((num, 3), dtype=int)

    row = 0
    for s, p, o in data_triples:
        if s in entities and o in entities:
            #s, o =  o2n[s], o2n[o]
            s,o = s,o
            nw.triples[row, :] = (s, p, o)
            row += 1

    nw.training = data_training.copy()
    for i in range(nw.training.shape[0]):
        nw.training[i, 0] = nw.training[i, 0] #o2n[nw.training[i, 0]]

    nw.withheld = data_withheld.copy()
    for i in range(nw.withheld.shape[0]):
        nw.withheld[i, 0] = nw.withheld[i, 0] #o2n[nw.withheld[i, 0]]

    nw.num_classes = data.num_classes

    nw.final = data.final
    nw.torch = data.torch
    if nw.torch:  # this should be constant-time/memory
        nw.triples = torch.from_numpy(nw.triples)
        nw.training = torch.from_numpy(nw.training)
        nw.withheld = torch.from_numpy(nw.withheld)

    return nw

In [ ]:
da_pruned = prunei(da,2)

In [ ]:
data = da_pruned
print(data.triples.shape)
data.entities = np.append(data.triples[:,0].detach().numpy(),(data.triples[:,2].detach().numpy()))
indices_nodes = data.entities
d = list(data.e2i.keys())
print(len(d), len(indices_nodes))
indices_nodes
values_indices_nodes = [d[i] for i in indices_nodes]

In [ ]:
data = da
print(data.triples.shape)
data.entities = np.append(data.triples[:,0].detach().numpy(),(data.triples[:,2].detach().numpy()))
indices_nodes = data.entities
d = list(data.e2i.keys())
indices_nodes
print(len(d), len(indices_nodes))
values_indices_nodes = [d[i] for i in indices_nodes]

In [ ]:
a = 0
for i in da_pruned.triples:
    if 5731 in i:
        print(i)
        a += 1
  


In [ ]:
a = 0
for i in da.triples:
    if 5731 in i:
        print(i)
        a += 1

In [ ]:
from rgcn_explainer_utils import *
edge_index = edge_index_oneadj(data.triples)
edge_index
find_n_hop_neighbors(edge_index,2,5757)

In [ ]:
imdb = torch.load("/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/RGCN_stuff/IMDb_typePeople_data.pt")

In [ ]:
imdb.num_relations

In [ ]:
imdb_pred = torch.load("pred_imdb_torch",map_location=torch.device('cpu'))
imdb_pred

In [ ]:
from rgcn_explainer_utils import prunee
data = kg.load('aifb', torch=True) 
data = prunee(data,2)
print(f'Number of entities: {data.num_entities}') #data.i2e
print(f'Number of classes: {data.num_classes}')
print(f'Types of relations: {data.num_relations}') #data.i2r
get_relations(data)

d = {key.item(): data.withheld[:, 0][data.withheld[:, 1] == key].tolist() for key in torch.unique(data.withheld[:, 1])}
# for i in d[0]:
#     print(data.i2e[i])
d

In [ ]:

from rgcn_explainer_utils import *
def selected(masked_ver, threshold,data, low_threshold):
    sel_masked_ver = sub_sparse_ten(masked_ver, threshold,data, low_threshold)
    indices_nodes = sel_masked_ver.coalesce().indices().detach().numpy()
    new_index = np.transpose(np.stack((indices_nodes[0], indices_nodes[1]))) 
    triples_matched = match_to_triples(np.array(new_index), data.triples)
    #print(triples_matched)
    l = []
    for i in triples_matched[:,1]:
        l.append(data.i2rel[int(i)][0])

    return Counter(l)

masked_ver = torch.load("/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/aifb_chk/masked_adj/masked_ver5757")
threshold = 0.5
low_threshold = False
sel_masked_ver = sub_sparse_tensor(masked_ver, threshold,data, low_threshold)
indices_nodes = sel_masked_ver.coalesce().indices().detach().numpy()
new_index = np.transpose(np.stack((indices_nodes[0], indices_nodes[1]))) 
new_index
s = selected(masked_ver, threshold,data, low_threshold)
# s

In [ ]:
s

In [ ]:
for i in data.triples:
    if 908 in i:
        print(i)


In [ ]:
triples_matched = match_to_triples(np.array(new_index), data.triples)
triples_matched

In [ ]:
def sub_sparse_ten(sparse_tensor, threshold, data, low_threshold=False):
    if low_threshold:
        nonzero_indices = sparse_tensor.coalesce().indices()[:, sparse_tensor.coalesce().values() < threshold]
        nonzero_indices[0] = nonzero_indices[0]%8285
        nonzero_values = sparse_tensor.coalesce().values()[sparse_tensor.coalesce().values() < threshold]
        sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
    else:
        nonzero_indices = sparse_tensor.coalesce().indices()[:, sparse_tensor.coalesce().values() > threshold]
        nonzero_indices[0] = nonzero_indices[0]%8285
        nonzero_values = sparse_tensor.coalesce().values()[sparse_tensor.coalesce().values() > threshold]
        sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)    
    return sel_masked_ver

In [ ]:
def match_to_triples(tensor1, tensor2):
    """
    tensor1: sub_edge tensor: edges of the neighborhood - transpose!!
    tensor2: data.triples: all edges
    """
    matching = []
    for i,i2 in zip(tensor1[:,0],tensor1[:,1]):
        for j,j1,j2, index in zip(tensor2[:,0],tensor2[:,1],  tensor2[:,2], range(len(tensor2[:,0]))):
            if i == j and i2 == j2:
                matching.append(tensor2[index])
                
    print(matching)
    # result = torch.stack(matching)
    # return result

# STEPS
1. edge index
2. Neighborhood
3. Extract neighborhood
4. Match to triples
5. Match to classes
6. Preprocess data: enrich, sum sparse, adj 
6. Get hor ver edges (adjacency for rels)
7. Edge mask
8. Masked hor ver edges
9. Train and save model and predictions
10. Loss function
11. Explain class - namely put all together
12. Forward and optimize for the masked adjacency


In [ ]:
#edge index
def edge_index_oneadj(triples):
    edge_index = torch.stack((triples[:, 0], triples[:, 2]),dim=0)
    return edge_index

#Neighborhood
#Extract neighborhood

def find_n_hop_neighbors(edge_index, n, node=None):
    """ 
    edge_index 
    n = num hops
    node = node_idx
    """
    # create dictionary of node neighborhoods
    neighborhoods = {}
    for i in range(edge_index.max().item() + 1):
        neighborhoods[i] = set()

    # find 1-hop neighbors and corresponding edges
    edges = []
    for j in range(edge_index.shape[1]):
        src, dst = edge_index[0, j].item(), edge_index[1, j].item()
        neighborhoods[src].add(dst)
        neighborhoods[dst].add(src)
        edges.append((src, dst))

    # find n-hop neighbors for the specified node or all nodes

    for k in range(2, n+1):
        new_neighbors = set()
        for neighbor in neighborhoods[node]:
            new_neighbors.update(neighborhoods[neighbor])
        neighborhoods[node].update(new_neighbors)
    sub_edges = []
    for edge in edges:
        src, dst = edge
        if src in neighborhoods[node] and dst in neighborhoods[node] or src == node or dst == node:
            sub_edges.append(edge)
            
    sub_edges_tensor = torch.tensor([sub_edges[i] for i in range(len(sub_edges))]).t()        

    #return {node: sub_edges}, {node: neighborhoods[node]}, sub_edges_tensor
    return sub_edges, neighborhoods[node], sub_edges_tensor


#match triples 
def match_to_triples(tensor1, tensor2):
    """
    tensor1: sub_edge tensor: edges of the neighborhood - transpose!!
    tensor2: data.triples: all edges
    """
    matching = []
    for i,i2 in zip(tensor1[:,0],tensor1[:,1]):
        for j,j1,j2, index in zip(tensor2[:,0],tensor2[:,1],  tensor2[:,2], range(len(tensor2[:,0]))):
            if i == j and i2 == j2:
                matching.append(tensor2[index])

    result = torch.stack(matching)
    return result



#PREPROCESS
def enrich(triples : torch.Tensor, n : int, r: int):
    """
    Enriches the given triples with self-loops and inverse relations.

    """
    cuda = triples.is_cuda

    inverses = torch.cat([
        triples[:, 2:],
        triples[:, 1:2] + r,
        triples[:, :1]
    ], dim=1)

    selfloops = torch.cat([
        torch.arange(n, dtype=torch.long,  device=d(cuda))[:, None],
        torch.full((n, 1), fill_value=2*r),
        torch.arange(n, dtype=torch.long, device=d(cuda))[:, None],
    ], dim=1)

    return torch.cat([triples, inverses, selfloops], dim=0)

def sum_sparse(indices, values, size, row=True):
    """
    Sum the rows or columns of a sparse matrix, and redistribute the
    results back to the non-sparse row/column entries

    :return:
    """

    ST = torch.cuda.sparse.FloatTensor if indices.is_cuda else torch.sparse.FloatTensor

    assert len(indices.size()) == 2

    k, r = indices.size()

    if not row:
        # transpose the matrix
        indices = torch.cat([indices[:, 1:2], indices[:, 0:1]], dim=1)
        size = size[1], size[0]

    ones = torch.ones((size[1], 1), device=d(indices))

    smatrix = ST(indices.t(), values, size=size)
    sums = torch.mm(smatrix, ones) # row/column sums

    sums = sums[indices[:, 0]]

    assert sums.size() == (k, 1)

    return sums.view(k)

def adj(triples, num_nodes, num_rels, cuda=False, vertical=True):
    """
     Computes a sparse adjacency matrix for the given graph (the adjacency matrices of all
     relations are stacked vertically).

     :param edges: List representing the triples
     :param i2r: list of relations
     :param i2n: list of nodes
     :return: sparse tensor
    """
    r, n = num_rels, num_nodes
    size = (r * n, n) if vertical else (n, r * n)

    from_indices = []
    upto_indices = []

    for s, p, o in triples:

        offset = p.item() * n
        

        if vertical:
            s = offset + s.item()
        else:
            o = offset + o.item()

        from_indices.append(s)
        upto_indices.append(o)
        

    indices = torch.tensor([from_indices, upto_indices], dtype=torch.long, device=d(cuda))


    assert indices.size(1) == len(triples)
    assert indices[0, :].max() < size[0], f'{indices[0, :].max()}, {size}, {r}'
    assert indices[1, :].max() < size[1], f'{indices[1, :].max()}, {size}, {r}'

    return indices.t(), size






#Get adjacency matrix: in this context this is hor / ver graph
def hor_ver_graph(triples, n, r):
    #triples = enrich(triples_small, n, r)

    hor_ind, hor_size = adj(triples, n, 2*r+1, vertical=False)
    ver_ind, ver_size = adj(triples, n, 2*r+1, vertical=True)
    #number of relations is 2*r+1 because we added the inverse and self loop

    _, rn = hor_size #horizontally stacked adjacency matrix size
    print(hor_size)
    r = rn // n #number of relations enriched divided by number of nodes

    vals = torch.ones(ver_ind.size(0), dtype=torch.float) #number of enriched triples
    vals = vals / sum_sparse(ver_ind, vals, ver_size) #normalize the values by the number of edges

    hor_graph = torch.sparse.FloatTensor(indices=hor_ind.t(), values=vals, size=hor_size) #size: n,r, emb


    ver_graph = torch.sparse.FloatTensor(indices=ver_ind.t(), values=vals, size=ver_size)

    return hor_graph, ver_graph


#Edge mask

def construct_edge_mask( num_nodes, init_strategy="normal", const_val=1.0):
    """
    Construct edge mask
    input;
        num_nodes: number of nodes in the neighborhood
        init_strategy: initialization strategy for the mask
        const_val: constant value for the mask
    output:
        mask: edge mask    
    """
    mask = nn.Parameter(torch.FloatTensor(num_nodes))  #initialize the mask
    if init_strategy == "normal":
        std = nn.init.calculate_gain("relu") * math.sqrt(
            2.0 / (num_nodes + num_nodes)
        )
        with torch.no_grad():
            mask.normal_(1.0, std)
    elif init_strategy == "const":
        nn.init.constant_(mask, const_val)
    return torch.tensor(mask)

#Masked 'adjacency' - masked hor vergraph

def _masked_adj(mask,adj, diag_mask, graph):
    """ Masked adjacency matrix 
    input: edge_mask, sub_adj, diag_mask
    output: masked_adj
    """
    sym_mask = mask
    sym_mask = torch.sigmoid(mask)
    
    sym_mask = (sym_mask + sym_mask.t()) / 2
    adj = torch.tensor(adj)
    masked_adj = adj * sym_mask

    #return masked_adj #* diag_mask
    return torch.sparse.FloatTensor(indices=graph.coalesce().indices(), values= masked_adj, size=graph.coalesce().size())



#match tro classes
def match_to_classes(tensor1, tensor2):
    """
    tensor1: sub graph indices
    tensor2: data.y labelsss
    """
    matching = []
    for i in (tensor1[:,0]):
        for j, index in zip(tensor2[:,0],range(len(tensor2[:,0]))):
            if i == j:
                matching.append(tensor2[index])
    return matching

## RGCN MODEL

In [ ]:
class RGCN(nn.Module):
    """
    Classic RGCN
    """

    def __init__(self, triples, n, r, numcls, emb=16, bases=None):

        super().__init__()

        self.emb = emb
        self.bases = bases
        self.numcls = numcls

        self.triples = enrich(triples, n, r)

        # horizontally and vertically stacked versions of the adjacency graph
        hor_ind, hor_size = adj(self.triples, n, 2*r+1, vertical=False)
        ver_ind, ver_size = adj(self.triples, n, 2*r+1, vertical=True)
        #number of relations is 2*r+1 because we added the inverse and self loop

        _, rn = hor_size #horizontally stacked adjacency matrix size
        r = rn // n #number of relations enriched divided by number of nodes

        vals = torch.ones(ver_ind.size(0), dtype=torch.float) #number of enriched triples
        vals = vals / sum_sparse(ver_ind, vals, ver_size) #normalize the values by the number of edges

        hor_graph = torch.sparse.FloatTensor(indices=hor_ind.t(), values=vals, size=hor_size) #size: n,r, emb
        
        
        self.register_buffer('hor_graph', hor_graph)

        ver_graph = torch.sparse.FloatTensor(indices=ver_ind.t(), values=vals, size=ver_size)
        self.register_buffer('ver_graph', ver_graph)

        # layer 1 weights
        if bases is None:
            self.weights1 = nn.Parameter(torch.FloatTensor(r, n, emb))
            nn.init.xavier_uniform_(self.weights1, gain=nn.init.calculate_gain('relu'))

            self.bases1 = None
        else:
            self.comps1 = nn.Parameter(torch.FloatTensor(r, bases))
            nn.init.xavier_uniform_(self.comps1, gain=nn.init.calculate_gain('relu'))

            self.bases1 = nn.Parameter(torch.FloatTensor(bases, n, emb))
            nn.init.xavier_uniform_(self.bases1, gain=nn.init.calculate_gain('relu'))

        # layer 2 weights
        if bases is None:

            self.weights2 = nn.Parameter(torch.FloatTensor(r, emb, numcls) )
            nn.init.xavier_uniform_(self.weights2, gain=nn.init.calculate_gain('relu'))

            self.bases2 = None
        else:
            self.comps2 = nn.Parameter(torch.FloatTensor(r, bases))
            nn.init.xavier_uniform_(self.comps2, gain=nn.init.calculate_gain('relu'))

            self.bases2 = nn.Parameter(torch.FloatTensor(bases, emb, numcls))
            nn.init.xavier_uniform_(self.bases2, gain=nn.init.calculate_gain('relu'))

        self.bias1 = nn.Parameter(torch.FloatTensor(emb).zero_())
        self.bias2 = nn.Parameter(torch.FloatTensor(numcls).zero_())

    def forward2(self, hor_graph, ver_graph):


        ## Layer 1

        n, rn = hor_graph.size() #horizontally stacked adjacency matrix size
        r = rn // n
        e = self.emb
        b, c = self.bases, self.numcls

        if self.bases1 is not None:
            # weights = torch.einsum('rb, bij -> rij', self.comps1, self.bases1)
            weights = torch.mm(self.comps1, self.bases1.view(b, n*e)).view(r, n, e)
        else:
            weights = self.weights1

        assert weights.size() == (r, n, e) #r relations, n nodes, e embedding size

        # Apply weights and sum over relations
        #hidden layer
        h = torch.mm(hor_graph, weights.view(r*n, e))  #matmul with horizontally stacked adjacency matrix and initialized weights
        assert h.size() == (n, e)

        h = F.relu(h + self.bias1) #apply non linearity and add bias

        ## Layer 2

        # Multiply adjacencies by hidden
        h = torch.mm(ver_graph, h) # sparse mm
        h = h.view(r, n, e) # new dim for the relations

        if self.bases2 is not None:
            # weights = torch.einsum('rb, bij -> rij', self.comps2, self.bases2)
            weights = torch.mm(self.comps2, self.bases2.view(b, e * c)).view(r, e, c)
        else:
            weights = self.weights2

        # Apply weights, sum over relations
        # h = torch.einsum('rhc, rnh -> nc', weights, h)
        h = torch.bmm(h, weights).sum(dim=0)

        assert h.size() == (n, c)

        return h + self.bias2 # -- softmax is applied in the loss
    



    def forward(self):


        ## Layer 1

        n, rn = self.hor_graph.size() #horizontally stacked adjacency matrix size
        r = rn // n
        e = self.emb
        b, c = self.bases, self.numcls

        if self.bases1 is not None:
            # weights = torch.einsum('rb, bij -> rij', self.comps1, self.bases1)
            weights = torch.mm(self.comps1, self.bases1.view(b, n*e)).view(r, n, e)
        else:
            weights = self.weights1

        assert weights.size() == (r, n, e) #r relations, n nodes, e embedding size

        # Apply weights and sum over relations
        #hidden layer
        h = torch.mm(self.hor_graph, weights.view(r*n, e))  #matmul with horizontally stacked adjacency matrix and initialized weights
        assert h.size() == (n, e)

        h = F.relu(h + self.bias1) #apply non linearity and add bias

        ## Layer 2

        # Multiply adjacencies by hidden
        h = torch.mm(self.ver_graph, h) # sparse mm
        h = h.view(r, n, e) # new dim for the relations

        if self.bases2 is not None:
            # weights = torch.einsum('rb, bij -> rij', self.comps2, self.bases2)
            weights = torch.mm(self.comps2, self.bases2.view(b, e * c)).view(r, e, c)
        else:
            weights = self.weights2

        # Apply weights, sum over relations
        # h = torch.einsum('rhc, rnh -> nc', weights, h)
        h = torch.bmm(h, weights).sum(dim=0)

        assert h.size() == (n, c)

        return h + self.bias2 # -- softmax is applied in the loss

    def penalty(self, p=2):
        """
        L2 penalty on the weights
        """
        assert p==2

        if self.bases is None:
            return self.weights1.pow(2).sum()

        return self.comps1.pow(p).sum() + self.bases1.pow(p).sum()


### EXPLAINER CLASS 
where we combine all of the previously defined functions for the sake of explanation training loop

loss function:

In [ ]:
#FUNCTIONS THAT WE USE IN THE EXPLAIN.PY FILE

def loss_fc(edge_mask,  pred, pred_label,label, node_idx, epoch, print=False):
    """
    Args:
        pred: y_e :  prediction made by current model
        pred_label: y_hat : the label predicted by the original model.
    """
    
    #PRED LOSS
    pred_label_node = pred_label[node_idx] #pred label is the prediction made by the original model
    gt_label_node = label[node_idx]

    logit = pred[gt_label_node] #pred is the prediction made by the current model

    pred_loss = -torch.log(logit) #this is basically taking the cross entropy loss

    # MASK SIZE EDGE LOSS
    
    mask = edge_mask
    mask = torch.sigmoid(mask)

    size_loss = 0.005 * torch.sum(mask)

    
    #MASK SIZE FEATURE LOSS
    # feat_mask = (torch.sigmoid(feat_mask))
    # feat_size_loss = 1.0 * torch.mean(feat_mask)

    # EDGE MASK ENTROPY LOSS
    mask_ent = -mask * torch.log(mask) - (1 - mask) * torch.log(1 - mask)
    mask_ent_loss = 1.0 * torch.mean(mask_ent)
    
    # FEATURE MASK ENTROPY LOSS
    # feat_mask_ent = - feat_mask * torch.log(feat_mask) - (1 - feat_mask) * torch.log(1 - feat_mask)

    # feat_mask_ent_loss = 0.1  * torch.mean(feat_mask_ent)

    # LAPLACIAN LOSS
    # D = torch.diag(torch.sum(masked_adj, 0))
    # m_adj = masked_adj 
    # L = D - m_adj

    # pred_label_t = torch.tensor(pred_label, dtype=torch.float)


    # lap_loss = ( 1.0
    #     * (pred_label_t @ L @ pred_label_t)
    #     / torch.Tensor(adj).numel())


    loss = pred_loss + size_loss  + mask_ent_loss # + feat_size_loss + lap_loss
    if print== True:
        print("optimization/size_loss", size_loss, epoch)
        #print("optimization/feat_size_loss", feat_size_loss, epoch)
        print("optimization/mask_ent_loss", mask_ent_loss, epoch)
        print(
            "optimization/feat_mask_ent_loss", mask_ent_loss, epoch
        )

        print("optimization/pred_loss", pred_loss, epoch)
        #print("optimization/lap_loss", lap_loss, epoch)
        print("optimization/overall_loss", loss, epoch)
    return loss




#SEE how the loss function is used in the explain.py file
# pred_label = torch.load('aifb_chk/prediction_aifb') #load prediction of original model
# pred_label = torch.argmax(pred_label[], dim=1) #get the prediction of the query node
# adj = torch.load('cora_chk/adj_cora') #load the adjacency matrix of the original model
# model = torch.load('cora_chk/model_cora') #load the original model

# #We are using the original model to get the prediction of the query node
# pred, adj = model.forward(torch.Tensor(subdata), torch.Tensor(sub_adj))
# node_pred = pred[node_idx_new, :] #get the prediction of the query node
# pred = nn.Softmax(dim=0)(node_pred) #apply softmax to it
# loss_fc( edge_mask, feat_mask, masked_adj,adj, pred, pred_label,data.y, node_idx, 1)

In [ ]:
from rgcn_explainer_utils import encode_classes, d_classes

In [ ]:
data = kg.load('aifb', torch=True) 
print(f'Number of entities: {data.num_entities}') #data.i2e
print(f'Number of classes: {data.num_classes}')
print(f'Types of relations: {data.num_relations}') #data.i2r



def get_relations(data):
    """ 
    This function is used to get the relations from the data.i2r dictionary and store it in a dictionary with the relation id as the key and the relation name as the value.
    """

    all_relations = []
    for i in range(data.num_relations):
        if '#' in str(data.i2r[i]).split('/')[3]:
            all_relations.append(str(data.i2r[i]).split('/')[3].split('#')[1])
        else:
            all_relations.append(str(data.i2r[i]).split('/')[3])
    dict = {}
    for i in range(len(all_relations)):
        dict[i] = [all_relations[i]]
        dict[i].append(data.i2r[i])

    data.i2rel = dict
    return data.i2rel
get_relations(data)
data.entities = np.append(data.triples[:,0].detach().numpy(),(data.triples[:,2].detach().numpy()))
get_relations(data)
d_classes(data)
#data.i2rel
#data.triples.shape

# data.withheld[:, 1]
# idxw, clsw = data.withheld[:, 0], data.withheld[:, 1]
# idxw, clsw = idxw.long(), clsw.long()
# d = list(data.e2i.keys())
#there is correspondance index and label person 
#print(dir(data))

In [ ]:
data.i2rel

In [ ]:
data.i2e[5757]

In [ ]:
pred_label = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/aifb_chk/prediction_aifb')
pred_label[1]
label = data.withheld[:, 1]

gt_label_node = label[1]


# logit = ypred[gt_label_node]
# -torch.log(logit) 

In [ ]:
class Explain(nn.Module):
    def __init__(self, model, data, node_idx, n_hops):
        super(Explain, self).__init__()
        #Those are the parameters of the original data and model
        self.model = model
        self.data = data
        self.n = data.num_entities
        self.r = data.num_relations
    
        #self.triples = enrich(data.triples, self.n, self.r)
        self.triples = data.triples
        self.node_idx = node_idx
        self.n_hops = n_hops
        #self.adj = get_adjacency(data)
        self.edge_index = edge_index_oneadj(self.triples)
        #print(self.edge_index)
        #self.label =  np.concatenate((np.array(data.training), np.array(data.withheld)), axis=0)
        self.label = self.data.withheld[:, 1]
        #self.feat = torch.Tensor(data.x)
        #self.feat_dim = data.num_features
        self.epoch = 1

        

        

        self.hor_graph, self.ver_graph = hor_ver_graph(self.triples, self.n, self.r)
        #print(self.hor_graph)
        self.pred_label = torch.load('aifb_chk/prediction_aifb')

        #self.node_idx_new, self.sub_adj, self.sub_feat, self.sub_label, self.neighbors = extract_neighborhood(self.node_idx, self.adj, self.feat, self.label, self.n_hops)
        self.sub_edges, self.neighbors, self.sub_edges_tensor = find_n_hop_neighbors(self.edge_index, n=self.n_hops, node=self.node_idx)
        #print('neighbors', self.neighbors)

        self.sub_triples = match_to_triples(self.sub_edges_tensor.t(),self.triples)
        #print('sub_triples', self.sub_triples)
        self.sub_hor_graph, self.sub_ver_graph = hor_ver_graph(self.sub_triples, self.n, self.r)
        self.num_nodes = self.sub_hor_graph.coalesce().values().shape[0] 
        
        #self.diag_mask = construct_diag_mask(self.neighbors)
        #self.subdata = torch.Tensor(data.subgraph(torch.tensor(self.neighbors)).x)
        self.edge_mask = construct_edge_mask(self.num_nodes)
        #print('edge_mask', self.edge_mask)
        #self.feat_mask = construct_feat_mask(self.feat_dim, init_strategy="normal")



    def _masked_adj(self, sub_graph):
        """ Masked adjacency matrix 
        input: edge_mask, sub_adj, diag_mask
        output: masked_adj
        """
        adj = sub_graph.coalesce().values()
        indices = sub_graph.coalesce().indices()
        size = sub_graph.coalesce().size()
        sym_mask = self.edge_mask
        sym_mask = torch.sigmoid(self.edge_mask)
        
        sym_mask = (sym_mask + sym_mask.t()) / 2
        adj = torch.tensor(adj)
        masked_adj = adj * sym_mask

        #return masked_adj #* diag_mask
        return torch.sparse.FloatTensor(indices=indices, values= masked_adj, size=size )
    
    def softmax(self, pred):
        """Compute softmax values for each sets of scores in x."""
        x_index = pred[self.node_idx].detach().numpy()
        e_x = np.exp(x_index - np.max(x_index))
        return e_x / e_x.sum(axis=0),  np.argmax(e_x / e_x.sum(axis=0))
    
    def new_index(self):
        idxw, clsw = self.data.withheld[:, 0], self.data.withheld[:, 1]
        idxw, clsw = idxw.long(), clsw.long()
        idxw_list = list(idxw)
        self.new_node_idx = idxw_list.index(self.node_idx)
        return self.new_node_idx 
        
    def forward(self):
        """
        Returns:
            ypred: prediction of the query node made by the current model (on the subgraph)

        """
        masked_hor = self._masked_adj(self.sub_hor_graph)
        masked_ver = self._masked_adj(self.sub_ver_graph)


        # self.masked_adj = self._masked_adj()
        # feat_mask = (torch.sigmoid(self.feat_mask))
        # x = self.sub_feat * feat_mask
        #ypred, adj_att = model(self.subdata, masked_adj)
        #ypred, adj_att = self.model(x, self.masked_adj)
        ypred = self.model.forward2(masked_hor, masked_ver)
 

        #_, res = self.softmax(ypred)
        
        node_pred = ypred[self.node_idx, :]
        res = nn.Softmax(dim=0)(node_pred)
  
        return res, masked_hor, masked_ver
    
    def criterion(self, epoch):
        """
        Computes the loss of the current model
        """
        #prediction of explanation model
        pred, masked_hor, masked_ver = self.forward()

        #prediction of original model
        #pred_label = self.pred_label[self.new_node_idx]

        self.new_node_idx = self.new_index()
        loss_val = loss_fc(self.edge_mask,  pred, self.pred_label,self.label, self.new_node_idx, epoch, print=False)

        return loss_val 
    
    # def mask_density(self):
    #     """
    #     Computes the density of the edge mask
    #     """
    #     mask_sum = torch.sum(self.masked_adj)
    #     adj_sum = torch.sum(self.adj)
    #     return mask_sum / adj_sum
    
    def return_stuff(self):
        #pred_label = torch.argmax(self.pred_label[self.neighbors], dim=1)
        return self.neighbors,self.n_hops, self.node_idx
    
    def adj_feat_grad(self, node_idx, pred_label_node):
        """
        Compute the gradient of the prediction w.r.t. the adjacency matrix
        and the node features.
        """
        self.model.zero_grad()
        self.masked_ver.requires_grad = True
        self.masked_hor.requires_grad = True
        
        if self.masked_hor.grad is not None:
            print('self.adj.grad', self.masked_hor.grad)
            self.masked_hor.grad.zero_() # zero out the gradient
            self.masked_ver.gradzero_() # zero out the gradient
        else:    
            masked_hor, masked_ver = self.masekd_hor, self.masked_ver
        ypred, _ = self.model(masked_hor, masked_ver)

        logit = nn.Softmax(dim=0)(ypred[node_idx, :])
        logit = logit[pred_label_node]
        loss = -torch.log(logit)
        loss.backward()
        return self.masked_hor.grad, self.masked_ver.grad
    
    def log_adj_grad(self, node_idx, pred_label, epoch, label=None):


        predicted_label = pred_label[node_idx]
        # adj_grad = torch.abs(self.adj_feat_grad(node_idx, predicted_label)[0])[self.graph_idx]
        masked_hor_grad, masekd_ver_grad = self.adj_feat_grad(node_idx, predicted_label)
    
        masked_hor_grad = torch.abs(masked_hor_grad)
        x_grad = x_grad[node_idx][:, np.newaxis]
            # x_grad = torch.sum(x_grad[self.graph_idx], 0, keepdim=True).t()
        masked_hor_grad = (masked_hor_grad + masked_hor_grad.t()) / 2
        masked_hor_grad = (masked_hor_grad * self.masked_hor).squeeze()
    
        # masked_adj = self.masked_adj[0].cpu().detach().numpy()

        # # only for graph mode since many node neighborhoods for syn tasks are relatively large for
        # # visualization
        
        # adj_grad = adj_grad.detach().numpy()
    

    


In [ ]:
explainer = Explain(model = torch.load('/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/aifb_chk/model_aifb'), data = data, node_idx = 5757, n_hops = 1)
optimizer = torch.optim.Adam(explainer.parameters(), lr=0.01)

In [ ]:
explainer.train()
for epoch in range(100):
    explainer.zero_grad()
    optimizer.zero_grad()
    ypred, masked_hor, masked_ver = explainer.forward()
    loss = explainer.criterion(epoch)
    neighbors,n_hops, node_idx = explainer.return_stuff()
    #pred_label,neighbors, , num_hops = explainer.return_stuff()
    



    loss.backward()
    optimizer.step()
    #mask_density = explainer.mask_density()
    # single_subgraph_label = sub_label.squeeze()
    # if epoch % 25 == 0:
    # #     explainer.log_mask(epoch)
    # #     explainer.log_masked_adj(
    # #         node_idx_new, epoch, label=single_subgraph_label
    # #     )


    #     explainer.log_adj_grad(
    #         node_idx_new, pred_label, epoch, label=single_subgraph_label)


    if epoch % 10 == 0:

        print(
        "epoch: ",
        epoch,
        "; loss: ",
        loss.item(),
        # "; mask density: ",
        # mask_density.item(),
        "; pred: ",
        ypred,
        # "; labels equal: ",
        # torch.argmax(ypred) == original_label== pred_label,

    )


# adj_atts = torch.sigmoid(adj_atts).squeeze()
# masked_adj = adj_atts * sub_adj.squeeze()
# masked_adj

In [ ]:
masked_ver.coalesce().values()

In [ ]:
indices_nodes = masked_ver.coalesce().indices().detach().numpy()

d = list(data.e2i.keys())
values_indices_nodes = [d[i] for i in indices_nodes[1]]
values_indices_nodes

np.transpose(indices_nodes)
new_ver = indices_nodes[0]%data.num_entities
indices_nodes[0]/data.num_entities #we indeed have 91 relations so that makes sense that max is around 90.5 sth relation
new_index = np.transpose(np.stack((new_ver, indices_nodes[1])))
new_index
G = nx.from_edgelist(new_index)
list(masked_ver.coalesce().values().detach().numpy())
G.number_of_edges()
values_indices_nodes

In [ ]:
indices_nodes = masked_ver.coalesce().indices().detach().numpy()

d = list(data.e2i.keys())
values_indices_nodes = [d[i] for i in indices_nodes[1]]
values_indices_nodes

In [ ]:
def dict_index_classes(data, masked_ver):
    indices_nodes = masked_ver.coalesce().indices().detach().numpy()

    d = list(data.e2i.keys())
    values_indices_nodes = [d[i] for i in indices_nodes[1]]
    dict = {}
    for i in range(len(values_indices_nodes)):
        try:
            dict[values_indices_nodes[i][0]] = str(values_indices_nodes[i][1]).split('/')[3]
            
        except IndexError :
            dict[values_indices_nodes[i][0]] = str(values_indices_nodes[i][1])
    return dict        
  
dict_index = dict_index_classes(data,masked_ver)
dict_index


In [ ]:
sub_triples = match_to_triples(new_index, data.triples)

In [ ]:
def dict_triples_semantics(data,  masked_ver, sub_triples):
    """ 
    Function to get a dictionary where the keys are the triples in indexes and values their semantic values
    Input:
        - Data
        - Sub Triples
        - Dict index
        
    """
    dict_index = dict_index_classes(data,masked_ver)
    indices_nodes = masked_ver.coalesce().indices().detach().numpy()
    new_ver = indices_nodes[0]%data.num_entities
    new_index = np.transpose(np.stack((new_ver, indices_nodes[1])))
    #sub_triples = match_to_triples(new_index, data.triples)
    dict = {}
    all_p = []
    for i in range(len(sub_triples)):

        s = dict_index[int(sub_triples[:,0][i])]
        p = data.i2rel[int(sub_triples[:,1][i])][0]
        o = dict_index[int(sub_triples[:,2][i])]
        all_p.append(p)
        dict[sub_triples[i]] = s + ' ' + p + ' ' + o 
    print(set(all_p))    
    return dict 

sem_triples = dict_triples_semantics(data, masked_ver, sub_triples)
sem_triples

In [ ]:
all_relations = []
for i in range(data.num_relations):
    if '#' in str(data.i2r[i]).split('/')[3]:
        all_relations.append(str(data.i2r[i]).split('/')[3].split('#')[1])
    else:
        all_relations.append(str(data.i2r[i]).split('/')[3])

all_relations 

In [ ]:
def get_relations(data):

    all_relations = []
    for i in range(data.num_relations):
        if '#' in str(data.i2r[i]).split('/')[3]:
            all_relations.append(str(data.i2r[i]).split('/')[3].split('#')[1])
        else:
            all_relations.append(str(data.i2r[i]).split('/')[3])
    dict = {}
    for i in range(len(all_relations)):
        dict[i] = [all_relations[i]]
        dict[i].append(data.i2r[i])

    data.i2rel = dict
    return data.i2rel


In [ ]:
# dict_index_classes(data,masked_ver)
# sub_triples
# #dict_index[int(sub_triples[:,0])]
# sub_triples[:,0][0]
# #dict_index[int(sub_triples[:,0][0])]
# dict_index
# len(sub_triples)
# len(neighbors)

In [ ]:
sparse_tensor = masked_ver
nonzero_indices = sparse_tensor.coalesce().indices()[:, sparse_tensor.coalesce().values() > 0.01]
nonzero_indices[0] = nonzero_indices[0]%data.num_entities
nonzero_values = sparse_tensor.coalesce().values()[sparse_tensor.coalesce().values() > 0.01]
nonzero_values
sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
sel_masked_ver

In [ ]:
a = masked_ver.coalesce().values()
b = sel_masked_ver.coalesce().values()
print(a, '\n', b)

In [ ]:
def sub_sparse_tensor(sparse_tensor, threshold):
    nonzero_indices = sparse_tensor.coalesce().indices()[:, sparse_tensor.coalesce().values() > threshold]
    nonzero_indices[0] = nonzero_indices[0]%data.num_entities
    nonzero_values = sparse_tensor.coalesce().values()[sparse_tensor.coalesce().values() > threshold]
    sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
    return sel_masked_ver
sub_sparse_tensor(masked_ver, 0.75)

In [ ]:


def sub_sparse_tensor(sparse_tensor, threshold, data, low_threshold=False):
    if low_threshold:
        nonzero_indices = sparse_tensor.coalesce().indices()[:, sparse_tensor.coalesce().values() < threshold]
        nonzero_indices[0] = nonzero_indices[0]%data.num_entities
        nonzero_values = sparse_tensor.coalesce().values()[sparse_tensor.coalesce().values() < threshold]
        sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
    else:
        nonzero_indices = sparse_tensor.coalesce().indices()[:, sparse_tensor.coalesce().values() > threshold]
        nonzero_indices[0] = nonzero_indices[0]%data.num_entities
        nonzero_values = sparse_tensor.coalesce().values()[sparse_tensor.coalesce().values() > threshold]
        sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)    
    return sel_masked_ver


def encode_classes(dict_index):
    d = []
    for k,v in dict_index.items():
        d.append(v)
    a = np.unique(d)
    dict = {}
    for i,j in zip(a, range(len(a))):
        dict[i] = j
        
    return dict


def encode_dict(dict_index):
    encoded_dict = {}
    dict = encode_classes(dict_index)
    for k,v in dict_index.items():
        for k1,v1 in dict.items():
            if v==k1:
                encoded_dict[k] = v1
                #encoded_dict[k].append(v)
    return encoded_dict



def visualize(node_idx, n_hop, data, masked_ver,threshold, result_weights=False, low_threshold=False ):
    """ 
    Visualize important nodes for node idx prediction
    """
    dict_index = dict_index_classes(data,masked_ver)
    
    
    #select only nodes with a certain threshold
    sel_masked_ver = sub_sparse_tensor(masked_ver, threshold,data, low_threshold)

    indices_nodes = sel_masked_ver.coalesce().indices().detach().numpy()
    new_index = np.transpose(np.stack((indices_nodes[0], indices_nodes[1]))) #original edge indexes

    
    
    G = nx.Graph()
    if result_weights:
        values = sel_masked_ver.coalesce().values().detach().numpy()
        for s,p,o in zip(indices_nodes[0],values , indices_nodes[1]):
            G.add_edge(int(s), int(o), weight=np.round(p, 2))

    else:
        #get triples to get relations 
        
        triples_matched = match_to_triples(np.array(new_index), data.triples)
        for s,p,o in triples_matched:
            G.add_edge(int(s), int(o), weight=int(p))

    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())


    pos = nx.circular_layout(G)

    ordered_dict = {}
    for item in list(G.nodes):
        if item in ordered_dict:
            ordered_dict[item].append(dict_index[item])
        else:
            ordered_dict[item] =  dict_index[item]

    dict_index = ordered_dict
    

    labeldict = {}
    for node in G.nodes:
        labeldict[int(node)] = int(node)  
    # color_list = list(encode_dict(dict_index).values())
    # print(color_list)

    dict = {}
    for k,v in dict_index.items():
        for k1,v1 in data.entities_classes.items():
            if v==k1: 

                dict[k] = v1
            else:
                if k not in dict:
                    dict[k] = 0
                

    color_list = list(dict.values())
    print(color_list)
    



    # color_dict = encode_classes(dict_index)
    # print(color_dict)

    
    if result_weights:
        
        nx.draw(G, pos,labels = labeldict,  edgelist=edges, edge_color=weights, node_color =  color_list, cmap="Set2",edge_cmap=plt.cm.Reds, font_size=7)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=nx.get_edge_attributes(G,'weight'),font_size=8,font_color='red')
        sm = plt.cm.ScalarMappable(cmap=plt.cm.Reds, norm=plt.Normalize(vmin=0, vmax=1))
        sm.set_array(weights)
        cbar = plt.colorbar(sm)
        cbar.ax.set_title('Weight')
        plt.title("Node {}'s {}-hop neighborhood important nodes".format(node_idx, n_hop))
        #plt.legend(data.entities_classes.values())
        plt.show()
    else:
        rel = nx.get_edge_attributes(G,'weight')
        for k,v in rel.items():
            rel[k] = data.i2rel[v][0]
        print(rel)
        nx.draw(G, pos,labels = labeldict,  edgelist=edges, edge_color=weights,node_color =  color_list, cmap="Set2",font_size=7)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=rel,font_size=5,font_color='red')
        # legend_colors = []
        # for k,v in set(color_dict.items()):
        #     legend_colors.append(plt.Line2D([0], [0], marker='o', color='w', label=k,
        #                         markerfacecolor=v, markersize=10))

        # # display the legend
        # plt.legend(handles= legend_colors)
        res = Counter(rel.values())
        plt.show()
        return res

    

In [ ]:
l = []
for i in sub_triples[:,1]:
    l.append(data.i2rel[int(i)][0])
print(Counter(l))

In [ ]:
mver_5797 = torch.load('aifb_chk/masked_ver5797')
mhor_5797 = torch.load('aifb_chk/masked_hor5797')

model = torch.load('aifb_chk/model_aifb')

ypred = model.forward2(mhor_5797, mver_5797)
node_pred = ypred[5797, :]
res = nn.Softmax(dim=0)(node_pred)
print(res)


In [ ]:
mver_5797.indices = mver_5797.coalesce().indices()[0]%data.num_entities
mver_5797.size()

In [ ]:
mver_5757 = torch.load('aifb_chk/masked_ver5757')
mhor_5757 = torch.load('aifb_chk/masked_hor5757')

model = torch.load('aifb_chk/model_aifb')


ypred = model.forward2(mhor_5757, mver_5757)
node_pred = ypred[5757, :]
res = nn.Softmax(dim=0)(node_pred)
print(res)

In [ ]:
#hor, ver = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
triples = enrich(data.triples,data.num_entities, data.num_relations)
hor, ver = hor_ver_graph(triples, data.num_entities, data.num_relations)

In [ ]:
hor, ver = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
model = torch.load('aifb_chk/model_aifb')

ypred = model.forward2(hor, ver)
ypred

In [ ]:
m = sub_sparse_tensor(masked_ver, 0, data, low_threshold=False)
a = match_to_triples(m.coalesce().indices().detach().numpy().T, data.triples)
l = []
for i in sub_triples[:,1]:
    l.append(data.i2rel[int(i)][0])
print(Counter(l))

In [ ]:
sub_triples

In [ ]:
from collections import Counter
rel = visualize(node_idx, 1, data, masked_ver, 0.5, result_weights= False, low_threshold=True)
rel

In [ ]:
def encode_dict(dict_index):
    encoded_dict = {}
    dict = encode_classes(dict_index)
    for k,v in dict_index.items():
        for k1,v1 in dict.items():
            if v==k1:
                encoded_dict[k] = v1
                #encoded_dict[k].append(v)
    return encoded_dict

encode_dict(dict_index)
encode_classes(dict_index)

In [ ]:
def encode_classes(dict_index):
    d = []
    for k,v in dict_index.items():
        d.append(v)
    a = np.unique(d)
    dict = {}
    for i,j in zip(a, range(len(a))):
        dict[i] = j
        
    return dict

dict_index = dict_index_classes(data,masked_ver)
a = encode_classes(dict_index)
dict_index

In [ ]:
dict = {}
for k,v in dict_index.items():
    for k1,v1 in data.entities_classes.items():
        if v==k1: 

            dict[k] = v1
        else:
            if k not in dict:
                dict[k] = 100
           

#color_list = list(dict.values())
dict


In [ ]:
def d_classes(data):
    """ 
    Get classes of nodes (select only the alphanum - not literals)
    """
    indices_nodes = data.entities
    d = list(data.e2i.keys())
    values_indices_nodes = [d[i] for i in indices_nodes]
    dict = {}
    for i in range(len(values_indices_nodes)):
        try:
            dict[values_indices_nodes[i][0]] = str(values_indices_nodes[i]).split('/')[3]
            
        except IndexError :
            dict[values_indices_nodes[i][0]] = str(values_indices_nodes[i])

    a = encode_classes(dict)   
    d = {}

    c = 0
    for k in a.keys():
        if k.isalpha():
            d[k] = c
            c+=1
    data.entities_classes = d
    return d      

d_classes(data)
# dict_index = d_classes(data)
# a = encode_classes(dict_index)

# d = {}

# c = 0
# for k in a.keys():
#     if k.isalpha():
#         d[k] = c
#         c+=1
# dict       

In [ ]:
def encode_classes(dict_index):
    d = []
    for k,v in dict_index.items():
        d.append(v)
    a = np.unique(d)
    dict = {}
    for i,j in zip(a, range(len(a))):
        dict[i] = j
        
    return dict


def encode_dict(dict_index):
    encoded_dict = {}
    dict = encode_classes(dict_index)
    for k,v in dict_index.items():
        for k1,v1 in dict.items():
            if v==k1:
                encoded_dict[k] = v1
    return encoded_dict

encode_dict(dict_index)

In [ ]:
#Visualize the result
def visualize_data(node_idx, data, num_hops):
    """Visualizes the n-hop neighborhood of a given node."""
    edge_index = edge_index_oneadj(data.triples)
    sub_edges, neighborhoods, sub_edges_tensor = find_n_hop_neighbors(edge_index, num_hops, node_idx)

    G = nx.from_edgelist(sub_edges)
    
    #create dict of index - node: to visualize index of the node
    labeldict = {}
    for node in G.nodes:
        labeldict[node] = node 
    print(G.nodes)
    print(G.number_of_edges)
    # dict_index = dict_index_classes(data,masked_ver)
    # #order dict index according to G nodes in networkx
    # ordered_dict = {}
    # for item in list(G.nodes):
    #     ordered_dict[item] = dict_index[item]

    # dict_index = ordered_dict
    

    # #get inverse of dict to allow mapping of different 'classes' of nodes to different nodes
    # inv_map = {v: k for k, v in dict_index.items()}
    # print(inv_map) #use inv:map to get a legend of node colors for later 
    # color_list = list(dict_index.values())
    
    # #make a list out of it 
    # for i in range(len(color_list)):
    #     if color_list[i] in inv_map:
    #         color_list[i] = inv_map[color_list[i]]   

            
    #edge colors reflect the masked ver values - more important relations have darker color
    #to check why we have relations than expexted :')

    #can get it through original triples 
    triples_matched = match_to_triples(np.array(sub_edges), data.triples)
    print(triples_matched)
    edge_colors = triples_matched[:,1].detach().numpy()
    print(edge_colors)
    colormap = plt.cm.ScalarMappable(norm=plt.Normalize(vmin=min(edge_colors), vmax=max(edge_colors)), cmap = "rainbow")

    # Create a color map for each value in the list
    colors = [colormap.to_rgba(val) for val in edge_colors]
    #cmap=cm.rainbow(np.array(edge_colors))


    # draw graph with edge colors
    plt.figure()  
    plt.title("Node {}'s {}-hop neighborhood".format(node_idx, num_hops))
    pos = nx.circular_layout(G)
    nx.draw(G, pos=pos, with_labels=True, edge_color = colors, edge_cmap=plt.cm.Reds,labels = labeldict, cmap="Set2" )



    plt.show()    

visualize_data(5797, data, 1)

In [ ]:
import torch
a = torch.load('aifb_chk/masked_ver1')
b = torch.load('aifb_chk/masked_ver2')
a, b = a.coalesce().values(), b.coalesce().values()
s = a.shape[0]
# print(s*s-np.count_nonzero(a==b))
# np.count_nonzero(b)
a,b = torch.tensor(a), torch.tensor(b)

res = torch.abs(a-b) <= 0.1
res.float().sum()

In [ ]:
res = torch.abs(a-b) <= 0.1
s*s- res.float().sum()
